# 4. Preventing overfitting

{TO-DO: Overview of notebook}

---

## Overfitting/Underfitting and Bias/Variance

- !! Explain what is overfitting and underfitting
    - Show images
- !! Explain Bias and Variance trade-off
    - Show images

## Training and testing data

It is good practice to test the generalizability of the fitted model to real-data generating mechanisms, a process sometimes called __model validation__. In this process we evaluate the performance of the model when predicting data that has never seen during training (but comes from the same distribution as the training data).

This leads us to the distinction between __training data__ and __testing data__. We use the training data to fit the parameters of the model. When we evaluate the performance of the model using the testing data, we leave these parameters fixed.

Let's see how we can split our data into training and testing set with _scikit-learn_, and use this process to illustrate the problem of overfitting. 

First, let's create a fake dataset for classification analysis. We will then fit and score a special kind of support vector classifier that uses a polinomial kernel of degree 4. You don't need to know what this means, besides that this type of model will be more complex and more flexible than a logistic regression model.

In [1]:
from sklearn.datasets import make_classification
from sklearn.svm import SVC

# Create fake dataset
X, y = make_classification(
    n_samples=100, n_features=20, n_informative=5, n_redundant=15, random_state=1
)

# Create and fit SVC
svc = SVC(kernel='poly', degree=4, random_state=0).fit(X, y)

# Score model
print(f"SVC mean performance: {svc.score(X, y)}")

SVC mean performance: 0.82


Notice that in the code cell above, and during all the previous examples of this tutorial, we have been using the same dataset for training and evaluating the performance of the model. As explained before, this can result in overestimation of the model perfomance.

Let's instead split our dataset into a training and testing set. We can do so using the function `train_test_split` in _scikit-learn_ (read the documentation [here](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)):

In [3]:
from sklearn.model_selection import train_test_split

# Split X and y into training and testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

Let's print the shapes of the training and testing set:

In [4]:
print(f"Shape of training set: {X_train.shape}")
print(f"Shape of training labels: {y_train.shape}")
print(f"Shape of testing sett: {X_test.shape}")
print(f"Shape of testing labels: {y_test.shape}")

Shape of training set: (75, 20)
Shape of training labels: (75,)
Shape of testing sett: (25, 20)
Shape of testing labels: (25,)


By default, _scikit-learn_ splits the dataset into a training set containing 75% of the original data, and a testing size containing the remaining 25%.

What happens if we fit our estimator to the training set, and evaluate it both using the training and testing set?

In [5]:
import numpy as np

# Fit model to training set
svc = SVC(kernel='poly', degree=5, random_state=0).fit(X_train, y_train)

# Evaluate model with training and testing data
## subsample training data for unbiased estimate
np.random.seed(0)
idx = np.random.choice(np.arange(len(X_train)), size=25)
## print scores
print(f"SVC mean accuracy on training data: {svc.score(X_train[idx], y_train[idx])}")
print(f"SVC mean accuracy on testing data: {svc.score(X_test, y_test)}")

SVC mean accuracy on training data: 0.84
SVC mean accuracy on testing data: 0.52


As you can see, the performance of the model drops significantly when evaluated on the testing set, indicating that our model may have overfitted to the training data.

#### ✍️ Exercise

Suppose we want our testing set size to comprise 20% of the original dataset. Modify `train_test_split` to achieve this aim. Write your answer in the cell below, and press the three dots to reveal the solution.

In [6]:
# Answer
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=20, random_state=0)

print(f"Proportion of testing set: {X_test.shape[0]/len(X)}")

Proportion of testing set: 0.2


----

Fortunately, there exists various methods that help reducing overfitting during training. In this notebook, we will revise two of them: _cross-validation_ and _regularization_.

# Cross-validation

One approach for performing model validation and prevent overestimating the real performance of the model is __cross validation__. In cross-validation we divide our sample dataset into different subsets. We then perform multiple rounds of training and validation (testing) of the model. In each round, we train on some of those subsets, and validate the model on the remaining ones. Importantly, the assignment of the subsets to training and testing sets rotates.

There are many types of cross-validation, but in this tutorial we will use as an example __Stratified K-Fold cross-validation__.


## Stratified K-Fold cross-validation

One popular type of cross-validation is __K-Fold__ cross validation. In K-Fold cross validation the dataset is split into $k$ equal subsets, also called folds. We then perform `k` rounds where each subset is used to validate the model while the others are used to train the data. Each subset is used for validation only once. This is better illustrated with the following image:

<figure>
  <img src="../images/kfoldcv.png" alt="kfoldcv" width="700"/>
  <figcaption>Image taken from https://sebastianraschka.com/faq/docs/evaluate-a-model.html </figcaption>
</figure>


Another type of cross-validation is __Stratified K-Fold__. It follows the same logic of K-Fold cross-validation, but with this technique we make sure that each fold of the dataset has the same proportion of samples for each class.

Let's implement Stratified K-Fold in scikit learn. We will first create some fake data for classification:

In [7]:
from sklearn.datasets import make_classification

# Create fake dataset
X, y = make_classification(
    n_samples=500, n_features=300, n_informative=100, random_state=0
)

Let's create a stratified cross-validation object using `StratifiedKFold` (read the documentation [here](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html#sklearn.model_selection.StratifiedKFold)). We will use 3 folds (defined here as `n_splits`):

In [8]:
from sklearn.model_selection import StratifiedKFold

# Create cross validation object
skf = StratifiedKFold(n_splits=3)

Let's create a pipeline chaining a standard scaler and a logistic regression model, and evaluate the performance of the model using the function `cross_validate` (read the documentation [here](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html) which takes as input the cross-validation object created above:

In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# Create pipeline
pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', LogisticRegression())
])

# Perform cross-validation
cv_results = cross_validate(
    pipe, X, y, scoring='accuracy', cv=skf
)

It's important to note that when we pass a pipeline to the `cross_validate` function, both the parameters of the transformer and the model remain fixed when transforming and evaluating the testing data. This prevents leaking data from the transformer to the test set.

Let's inspect the output of the cross-validation procedure:

In [14]:
cv_results

{'fit_time': array([0.01996017, 0.02074409, 0.02287984]),
 'score_time': array([0.00092697, 0.00080991, 0.00081515]),
 'test_score': array([0.74251497, 0.69461078, 0.60843373])}

Most often these test scores are averaged and reported as the final test score. We can do so by running the following:

In [15]:
import numpy as np

# Compute mean of test scores
mean_test_score = cv_results["test_score"].mean()

# Print mean test score
print(f"Mean test score: {np.round(mean_test_score, 2)}")

Mean test score: 0.68


We can also return the training scores and estimators used in each split by setting to `True` the parameters `return_train_score` and `return_estimator`:

In [17]:
cv_results = cross_validate(
    pipe, X, y, scoring='accuracy', cv=skf,
    return_train_score=True,
    return_estimator=True
)

In [18]:
cv_results

{'fit_time': array([0.0154357 , 0.01691628, 0.01284504]),
 'score_time': array([0.0005281 , 0.00153494, 0.00158811]),
 'estimator': [Pipeline(steps=[('scaler', StandardScaler()), ('clf', LogisticRegression())]),
  Pipeline(steps=[('scaler', StandardScaler()), ('clf', LogisticRegression())]),
  Pipeline(steps=[('scaler', StandardScaler()), ('clf', LogisticRegression())])],
 'test_score': array([0.74251497, 0.69461078, 0.60843373]),
 'train_score': array([1., 1., 1.])}

Note that cross validation does not prevent overfitting to the training set, but rather reduces the bias when estimating and reporting the performance of our model.

#### ✍️ Exercise

Can you read the documentation of `RepeatedStratifiedKFold` [here](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RepeatedStratifiedKFold.html#sklearn.model_selection.RepeatedStratifiedKFold), reflect on how it differs from `StratifiedKFold`, and implement it to train a model on our fake dataset? Write your answer in the cell below, and press the three dots to reveal the solution.

In [ ]:
# Answer
from sklearn.model_selection import RepeatedStratifiedKFold

# Create cross-validation
rsf = RepeatedStratifiedKFold(n_splits=10, n_repeats=10, random_state=0)

# Create model
clf = LogisticRegression()

# Cross validate
cv_results = cross_validate(
    clf, X, y, scoring='accuracy', cv=skf
)
cv_results

# Regularization

(...) Add description to this section

One way of reducing overfitting is to regularize our models. Regularization penalizes to model to avoid it from over-adjusting to the training set. The penalty term is added to the _loss_ function of the model, and the way it is defined determines the type of regularization.

The most well-known regularization types are:
- __L1 or _Lasso___: Makes the coefficients of the model sparse, meaning some of them are shrinked to 0. This results in the model looking simpler.
- __L2 or _Ridge___: Makes the coefficients of the model smaller. This results in the model looking smoother.

In this tutorial we won't provide the mathematical details behind these types of regularization, but you can read them [here])

Let's explore how _scikit-learn_ implements regularization in a logistic regression. Let's create such model and inspect its parameters:

In [19]:
from sklearn.linear_model import LogisticRegression

# Create model
clf = LogisticRegression()
vars(clf)

{'penalty': 'l2',
 'dual': False,
 'tol': 0.0001,
 'C': 1.0,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'class_weight': None,
 'random_state': None,
 'solver': 'lbfgs',
 'max_iter': 100,
 'multi_class': 'auto',
 'verbose': 0,
 'warm_start': False,
 'n_jobs': None,
 'l1_ratio': None}

As you can see, by default scikit-learn penalizes logistic regression models using _Ridge (L2)_ regularization. If you read the documentation of `LogisticRegression` [here](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) you will also notice that the parameter `C` determines the strenght of the regularization applied. By default this value is set to 1. Lower values will indicate stronger regularization.

Let's know explore how regularizing impacts the values of the coefficients. Using an _L2_ regularization, let's determine how the sum of the absolute value of the coefficients increases with less regularization:

In [20]:
# List of regularization values to be explored
c_values = [0.001, 0.1, 100]

# Print sum of absolute values of coefficients
for c_val in c_values:
    clf = LogisticRegression(penalty="l2", C=c_val).fit(X_train, y_train)
    print(f"Sum of coefficients with C {c_val}: {np.sum(np.abs(clf.coef_))}")

Sum of coefficients with C 0.001: 0.34188132500646984
Sum of coefficients with C 0.1: 3.201367995215172
Sum of coefficients with C 100: 5.378647816301111


As expected, the values get bigger with less regularization. What happens when we use _L1_ regularization? Think about it before looking at the code cell below.

In [21]:
# Print number of coefficients set to 0
for c_val in c_values:
    clf = LogisticRegression(penalty="l1", solver="liblinear", C=c_val).fit(X_train, y_train)
    zero_coef = np.sum((clf.coef_)==0)
    print(f"Number of coefficients set to 0 with C {c_val}: {zero_coef}")

Number of coefficients set to 0 with C 0.001: 20
Number of coefficients set to 0 with C 0.1: 18
Number of coefficients set to 0 with C 100: 12


As expected, increasing the strength of _L1_ regularization increases the number of coefficients that are set to 0 when training the model.

#### ✍️ Exercise

Can you explore the [API](https://scikit-learn.org/stable/modules/classes.html) of _scikit-learn_ and determine how a create linear regression model using _L2_ (also called _Ridge_) regularization? Write your answer in the cell below, and press the three dots to reveal the solution.

_Hint:_ It will not be able to add regularization in the same way as with a logistic regression model. Check the module [sklearn.linear_model](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.linear_model) for ideas on how to implement it instead.

__Answer__

A linear regression model using _l2_ regularization can be created as follows:
```python

from sklearn.linear_model import Ridge
ridge = Ridge()

```


# _Optional material_: Hyper-parameter tuning

The value of regularization that gives the best performance in not known beforehand. Some people search the value of regularization that gives the best performance value manually, using the whole training and testing dataset. This could lead to __selection bias__, which in turns results in overestimating the real performance of the model. Instead, good practice would be to tune this parameter the same as the weights of the model. This leads us to understanding the concept of __hyper-parameter tuning__.

(...) add description to this section
- What are hyperparameters
    - don't change during training the coefficients of the model
- Why we should tune our hyperparametes using cross validation and not search them manually.
    - concept of __selection bias__
- Concept of Nested Cross Validation
    - Image {see https://sebastianraschka.com/faq/docs/evaluate-a-model.html}
- What is, and how to perform, `GridSearchCV`

In [22]:
from sklearn.datasets import make_moons

# Create fake dataset
X, y = make_moons(noise=0.352, random_state=1, n_samples=100)

In [ ]:
from sklearn.model_selection import GridSearchCV, RepeatedStratifiedKFold
from sklearn.svm import SVC

param_grid = [
    {'kernel': ['linear']},
    {'kernel': ['poly'], 'degree': [2, 3]},
    {'kernel': ['rbf']}
]

svc = SVC(random_state=0)

cv = RepeatedStratifiedKFold(
    n_splits=10, n_repeats=10, random_state=0
)

search = GridSearchCV(
    estimator=svc, param_grid=param_grid,
    scoring='roc_auc', cv=cv
)
search.fit(X, y)

In [ ]:
import pandas as pd

results_df = pd.DataFrame(search.cv_results_)
results_df

In [ ]:
results_df = results_df.sort_values(by=['rank_test_score'])
results_df = (
    results_df
    .set_index(results_df["params"].apply(
        lambda x: "_".join(str(val) for val in x.values()))
    )
    .rename_axis('kernel')
)
results_df[
    ['params', 'rank_test_score', 'mean_test_score', 'std_test_score']
]

Learn how to perform statistical evaluation over these examples [here](https://scikit-learn.org/stable/auto_examples/model_selection/plot_grid_search_stats.html)

# ✏️ Check your knowledge
Load the ABIDE 2 dataset and:

1. Perform classification analyses using `StratifiedKFold` with 10 folds.
    - How variable are the testing scores?
    - How different is the accuracy obtained in the training set as compared to the testing set?
    - Compute the mean accuracy over folds to obtain a final estimate of the performance of the model.
2. If you have read the optional material, use `GridSearchCV` to determine which regularization technique (_L1_ or _L2_) and value of `C` gives the best performance when using `SVC` to perform classification analysis.


# Additional resources

- [Overfitting and underfitting](https://github.com/neurohackademy/nh2020-curriculum/blob/master/tu-machine-learning-yarkoni/03-overfitting.ipynb), [Validation](https://github.com/neurohackademy/nh2020-curriculum/blob/master/tu-machine-learning-yarkoni/04-validation.ipynb) and [Model selection](https://github.com/neurohackademy/nh2020-curriculum/blob/master/tu-machine-learning-yarkoni/05-model-selection.ipynb) by _Tal Yarkoni_.